In [ ]:
import os, sys, time

import requests
import shutil

# https://gitlab.com/dlr-ve-esy/esy-osmfilter/-/tree/master/
from esy.osmfilter import  osm_colors          as CC
from esy.osmfilter import run_filter, Node,Way,Relation 
from esy.osmfilter import export_geojson
from contextlib import contextmanager

import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
# import logging
# logging.basicConfig()
# logger=logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
# logger.setLevel(logging.WARNING)


In [ ]:
country_name = 'africa' # TODO: Create dictionary with country_code : country_name pairs
country_code = 'AFRICA'
# country_name = 'cape-verde'
geofabrik_filename=f'{country_name}-latest.osm.pbf' # Filename for geofabrik 
geofabrik_url = f'https://download.geofabrik.de/africa/{geofabrik_filename}' # https://download.geofabrik.de/africa/nigeria-latest.osm.pbf

local_file= os.path.join("data","osm",geofabrik_filename)
if not os.path.exists(local_file):
    print(f"{geofabrik_filename} does not exist, downloading to {local_file}")
    os.makedirs(os.path.dirname(local_file), exist_ok=True) # create data/osm directory
    with requests.get(geofabrik_url, stream=True) as r:
        with open(local_file, 'wb') as f:
            shutil.copyfileobj(r.raw, f)

# Input filepath
PBF_inputfile = os.path.join(os.getcwd(),local_file)

# json file for the Data dictionary 
# json file for the Elements dictionary is automatically written to 'data/osm/Elements'+filename)
JSON_outputfile = os.path.join(os.getcwd(),'data','osm',country_name+'_substations.json')



In [ ]:
print(PBF_inputfile, JSON_outputfile)

In [ ]:
#see https://dlr-ve-esy.gitlab.io/esy-osmfilter/filter.html for filter structures

prefilter = {Node: {"power":["substation"]}, Way: {"power":["substation"]}, Relation: {"power":["substation"]}}
blackfilter = [("pipeline","substation"),]
whitefilter =[[("power","substation"),],]
    
elementname=f'{country_code}_substations'

[Data,Elements]=run_filter(elementname,PBF_inputfile, JSON_outputfile,prefilter,whitefilter,blackfilter, NewPreFilterData=False, CreateElements=False, LoadElements=True,verbose=False,multiprocess=True)


# export the elements to a geojson-file
# export_geojson(Elements[elementname]['Way'],Data,filename='test.geojson',jsontype='Line')
# convert extract node elements
# export_geojson(Elements[elementname]['Node'],Data,filename='test.geojson',jsontype='Point')




In [ ]:
import pickle
Elements = pickle.load(os.path.join(os.getcwd(),os.path.dirname(JSON_outputfile), 'Elements', 'AFRICA_substations.pickle'))

# Elements = osm_pickle.pickleload(Elements,os.path.join(os.getcwd(),os.path.dirname(JSON_outputfile), 'Elements/'))    

In [ ]:
df = pd.json_normalize(Elements[elementname]["Way"].values())
df = df.dropna(subset=['tags.voltage'])
df = df.dropna(axis=1, how='all')

display(df)



In [ ]:
df1 = pd.json_normalize(Elements[elementname]["Node"].values())
df1 = df1.dropna(axis=1, how='all')

display(df1)


In [ ]:
lonlat_column =[]
for ref in df["refs"]:
    lonlats =[]
    for r in ref:
        lonlat = Data["Node"][str(r)]["lonlat"]
        lonlats.append(lonlat)
    lonlats = np.array(lonlats)
    lonlat = np.mean(lonlats, axis=0)
    lonlat_column.append(lonlat)
# display(lonlat_column)
df["lonlat"] = lonlat_column

display(df)
    
# polygon_column=[]
# for poly_list in lonlat_column:
#     for p in poly_list:
#         print(p)

#     print("p")
    

In [ ]:
gdf = gpd.GeoDataFrame(f, geometry=[Point(x, y) for x, y in df.lonlat])

In [ ]:
display(gdf)

In [ ]:
gdf = gdf.drop(columns=['lonlat'])
gdf = gdf.drop(columns=['refs'])
gdf.to_file(f"./{countr_cod}.geojson", driver="GeoJSON")  

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
#world
saudi = world.query('name == "Saudi Arabia"')
yemen = world.query('name == "Yemen"')
iraq = world.query('name == "Iraq"')
oman = world.query('name == "Oman"')
uae = world.query('name == "United Arab Emirates"')
jordan = world.query('name == "Jordan"')
palestine = world.query('name == "Palestine"')
israel = world.query('name == "Israel"')
qatar = world.query('name == "Qatar"')
kuwait = world.query('name == "Kuwait"')

mena = saudi.append(yemen).append(iraq).append(oman).append(uae).append(jordan).append(palestine).append(israel).append(qatar).append(kuwait)

africa = world.query('continent == "Africa"').append(mena)
#africa.head()

In [ ]:
import geoplot
nigeria = gpd.read_file('./my.geojson')
# url = 'https://development-data-hub-s3-public.s3.amazonaws.com/ddhfiles/144823/africagrid20170906final.geojson'
# africa_network = gpd.read_file(url)
ax = nigeria.plot(figsize=(20, 20),alpha = 0.7)
geoplot.pointplot(nigeria, edgecolor='gray', ax=ax)
geoplot.polyplot(africa, edgecolor='gray', ax=ax)